<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/6_2_chatbot_external_memory_with_postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with message summarization & external DB memory

## Review

We've covered how to customize graph state schema and reducer.

We've also shown a number of tricks for trimming or filtering messages in graph state.

We've used these concepts in a Chatbot with memory that produces a running summary of the conversation.

## Goals

But, what if we want our Chatbot to have memory that persists indefinitely?

Now, we'll introduce some more advanced checkpointers that support external databases.

Here, we'll show how to use [Postgres as a checkpointer](https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/)

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai


In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
import os
os.environ["Langchain_api_key"] = userdata.get('Langchain_api_key')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

# Use sync connection¶
This sets up a synchronous connection to the database.

Synchronous connections execute operations in a blocking manner, meaning each operation waits for completion before moving to the next one. The DB_URI is the database connection URI, with the protocol used for connecting to a PostgreSQL database, authentication, and host where database is running. The connection_kwargs dictionary defines additional parameters for the database connection.

In [ ]:
from google.colab import userdata
DB_URI = userdata.get('DB_URI')

In [ ]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up


Let's re-define our chatbot.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  GEMINI_API_KEY)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

Now, we just re-compile with our postgres checkpointer.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)


Now, we can invoke the graph several times.

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="hi! I'm hafiz naveed uddin")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: The conversation began with Wania introducing herself.  She then stated that she enjoys painting.  We discussed her hobby briefly, and finally, she asked me to summarize our conversation, which I am now doing.



Message count: 4
================================== Ai Message ==================================

Hi Hafiz Naveed Uddin!  It's nice to meet you. How can I help you today?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Wania", additional_kwargs={}, response_metadata={}, id='8f12d944-d396-452f-b5e5-facd70a69850'), AIMessage(content="Hi Wania!  It's nice to talk to you again.  How's your painting going?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-dd5e0bca-b09f-47a8-bea6-d6db87a01377-0', usage_metadata={'input_tokens': 106, 'output_tokens': 24, 'total_tokens': 130, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="hi! I'm hafiz naveed uddin", additional_kwargs={}, response_metadata={}, id='414753fd-174c-4d6c-aefe-4d3ff9b954b5'), AIMessage(content="Hi Hafiz Naveed Uddin!  It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f58b14cd-5bb3-4d50-a3a4-e334

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="I like painting pictures.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: The conversation began with Wania introducing herself.  She then stated that she enjoys painting.  We discussed her hobby briefly, and finally, she asked me to summarize our conversation, which I am now doing.



Message count: 6
================================== Ai Message ==================================

That's wonderful! What kind of pictures do you enjoy painting?  Do you have a favorite medium (like oils, watercolors, acrylics)?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Wania", additional_kwargs={}, response_metadata={}, id='8f12d944-d396-452f-b5e5-facd70a69850'), AIMessage(content="Hi Wania!  It's nice to talk to you again.  How's your painting going?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-dd5e0bca-b09f-47a8-bea6-d6db87a01377-0', usage_metadata={'input_tokens': 106, 'output_tokens': 24, 'total_tokens': 130, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="hi! I'm hafiz naveed uddin", additional_kwargs={}, response_metadata={}, id='414753fd-174c-4d6c-aefe-4d3ff9b954b5'), AIMessage(content="Hi Hafiz Naveed Uddin!  It's nice to meet you. How can I help you today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f58b14cd-5bb3-4d50-a3a4-e334

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="What's my name and what is my hobby?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: The conversation began with Wania introducing herself.  She then stated that she enjoys painting.  We discussed her hobby briefly, and finally, she asked me to summarize our conversation, which I am now doing.



Message count: 8
Messages before summarizing: 8
Existing summary: The conversation began with Wania introducing herself.  She then stated that she enjoys painting.  We discussed her hobby briefly, and finally, she asked me to summarize our conversation, which I am now doing.



New summary: The conversation began with Wania introducing herself and stating her enjoyment of painting.  This was followed by Hafiz Naveed Uddin introducing himself and also declaring his fondness for painting pictures.  A brief discussion about painting ensued, culminating in Hafiz Naveed Uddin asking for a summary of the conversation up to that point.  This summary then incorporates the additional messages, demonstrating the chatbot's ability to maintain context and respond appropriately to requests for information.

Messages after truncation: 6
================================== Ai Message ==================================

Your name is Hafiz Naveed Uddin, and your hobby is painting pictures.


StateSnapshot(values={'messages': [HumanMessage(content="What's my name and what is my hobby?", additional_kwargs={}, response_metadata={}, id='342921f1-283e-4371-80fa-8ad30e37d3a4'), AIMessage(content='Your name is Hafiz Naveed Uddin, and your hobby is painting pictures.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5fc041b8-b24a-4529-8568-f06aa0b6967b-0', usage_metadata={'input_tokens': 170, 'output_tokens': 18, 'total_tokens': 188, 'input_token_details': {'cache_read': 0}})], 'summary': "The conversation began with Wania introducing herself and stating her enjoyment of painting.  This was followed by Hafiz Naveed Uddin introducing himself and also declaring his fondness for painting pictures.  A brief discussion about painting ensued, culminating in Hafiz Naveed Uddin asking for a summary of the conversation up to that point.  This summary then incorporates the additi

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="Can you describe about abstract paintings?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: The conversation began with Wania introducing herself and stating her enjoyment of painting.  This was followed by Hafiz Naveed Uddin introducing himself and also declaring his fondness for painting pictures.  A brief discussion about painting ensued, culminating in Hafiz Naveed Uddin asking for a summary of the conversation up to that point.  This summary then incorporates the additional messages, demonstrating the chatbot's ability to maintain context and respond appropriately to requests for information.



Message count: 4
================================== Ai Message ==================================

Abstract paintings are works of art that don't depict recognizable objects or scenes from the real world. Instead, they focus on expressing ideas and emotions through shapes, colors, forms, and textures.  There's no single definition, as the style encompasses a wide range of approaches.

Here are some key characteristics of abstract paintings:

* **Non-representational:**  They don't aim to represent something specific from reality.  Instead, they use visual elements to create a feeling or atmosphere.
* **Emphasis on form and color:**  The arrangement of shapes, lines, and colors is paramount.  The interplay between these elements is what conveys meaning.
* **Emotional expression:**  Many abstract artists use their work to express emotions or ideas that are difficult to articulate through words.
* **Experimentation:**  Abstract art often involves experimentation with different techniques 

StateSnapshot(values={'messages': [HumanMessage(content="What's my name and what is my hobby?", additional_kwargs={}, response_metadata={}, id='342921f1-283e-4371-80fa-8ad30e37d3a4'), AIMessage(content='Your name is Hafiz Naveed Uddin, and your hobby is painting pictures.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5fc041b8-b24a-4529-8568-f06aa0b6967b-0', usage_metadata={'input_tokens': 170, 'output_tokens': 18, 'total_tokens': 188, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Can you describe about abstract paintings?', additional_kwargs={}, response_metadata={}, id='9761af59-dadc-4cc1-9318-9fd25be6d043'), AIMessage(content="Abstract paintings are works of art that don't depict recognizable objects or scenes from the real world. Instead, they focus on expressing ideas and emotions through shapes, colors, forms, and textures.  There's no single def

In [ ]:
# Retrieve state using thread ID
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content="What's my name and what is my hobby?", additional_kwargs={}, response_metadata={}, id='342921f1-283e-4371-80fa-8ad30e37d3a4'), AIMessage(content='Your name is Hafiz Naveed Uddin, and your hobby is painting pictures.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5fc041b8-b24a-4529-8568-f06aa0b6967b-0', usage_metadata={'input_tokens': 170, 'output_tokens': 18, 'total_tokens': 188, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Can you describe about abstract paintings?', additional_kwargs={}, response_metadata={}, id='9761af59-dadc-4cc1-9318-9fd25be6d043'), AIMessage(content="Abstract paintings are works of art that don't depict recognizable objects or scenes from the real world. Instead, they focus on expressing ideas and emotions through shapes, colors, forms, and textures.  There's no single def

In [ ]:
pool.close()



### Persisting state

Using database like Postgres means state is persisted!

For example, we can re-start the notebook kernel and see that we can still load from Postgres DB on disk.
